<a href="https://colab.research.google.com/github/shruti-singh/AdvMachineLearning/blob/main/assignment1/Q2A_CharRNNForTextGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# # Mount drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
cd /content/drive/My Drive/Coursework/Sem3/AdvanceML/HW1/

/content/drive/My Drive/Coursework/Sem3/AdvanceML/HW1


**Load the abstract data of NeurIPS and ICLR papers**

Dataset details in Q2B_1_TrainDistilBERTForTextGen.ipynb

Dataset construction in Q2B_0_create_training_data_small.ipynb

In [4]:
data_file_path = tf.keras.utils.get_file('/content/drive/My Drive/Coursework/Sem3/AdvanceML/HW1/training_data_paper_abstracts_neurips_iclr.txt', 'https://dummyurl.txt')

In [8]:
#Dataset
!head -10 ./training_data_paper_abstracts_neurips_iclr.txt

Solving a Class of Non-Convex Min-Max Games Using Iterative First Order Methods
Recent applications that arise in machine learning have surged significant interest in solving min-max saddle point games.
This problem has been extensively studied in the convex-concave regime for which a global equilibrium solution can be computed efficiently.
In this paper, we study the problem in the non-convex regime and show that an ε–first order stationary point of the game can be computed when one of the player’s objective can be optimized to global optimality efficiently.
In particular, we first consider the case where the objective of one of the players satisfies the Polyak-Łojasiewicz (PL) condition.
For such a game, we show that a simple multi-step gradient descent-ascent algorithm finds an ε–first order stationary point of the problem in Õ(ε−2) iterations.
Then we show that our framework can also be applied to the case where the objective of the “max-player" is concave.
In this case, we propose

In [10]:

print("\n\n\nDataset consists of research papers abstracts downloaded from NeurIPS (1987-2019) and ICLR (2013-2020). Data download script is attached separately.\n\n")




Dataset consists of research papers abstracts downloaded from NeurIPS (1987-2019) and ICLR (2013-2020). Data download script is attached separately.




In [6]:
text = open(data_file_path, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 17142750 characters


In [12]:
char_vocab = sorted(set(text))

In [13]:
print("Char vocab length: ", len(char_vocab))

Char vocab length:  384


In [17]:
print(char_vocab[50:100])

['G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


As we can see, there are a lot of special characters in the research paper text, which are usually absent in the daily usage text/literature. 
Research papers contain some greek characters such as lambda etc, and all sorts of mathematical symbols, which makes text generation for research papers harder.

In [19]:
# Creating a vocan mapping
char_idx_map = {u:i for i, u in enumerate(char_vocab)}
idx_char_map = np.array(char_vocab)

# To be fed into the model
text_as_integers = np.array([char_idx_map[c] for c in text])

In [20]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_integers)

In [21]:
# Truncate sequences
seq_length = 512
examples_per_epoch = len(text)//(seq_length+1)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


For char level models, the output is one char left shifted version of input, as the goal is to predict the next character.

In [22]:
def convert_input_to_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(convert_input_to_target)

In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
class CharLevelRNN:
    def __init__(self, char_vocab):
        self.vocab_size = len(char_vocab)
        self.embedding_dim = 256
        self.rnn_units = 1024
        self.batch_size = BATCH_SIZE

        self.model = None
        self.epochs = 10
        self.optimizer = 'adam'
        return
    
    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
    def train_model(self, dataset):
        self.model = tf.keras.Sequential([tf.keras.layers.Embedding(self.vocab_size,  self.embedding_dim, batch_input_shape=[batch_size, None]),
                                     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.rnn_units)),
                                     tf.keras.layers.Dense(vocab_size)])
        self.model.compile(optimizer=self.optimizer, loss=loss)
        history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
        return
    
    def get_model_summary(self):
        print(self.model.summary())
        return

**References**